<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/spell/S_spell_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 기본		    G_inc_again_so_model.pt
 산업분류자동화    'My Drive'
'Colab Notebooks'   Screenshot_20220401-001411_Chrome.jpg


In [3]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/산업분류자동화/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")

## 데이터 전처리

In [4]:
df=df[(df['digit_1'] =='S')] #조건에 맞는 데이터 출력

In [5]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환
1,id_0000017,S,95,952,사업장에서,고객요청으로,자동차세차업
2,id_0000029,S,96,961,미용실에서,일반인대상,컷.파마
3,id_0000034,S,96,969,세탁소에서,고객의 의뢰를 받아,가정용 세탁물 세탁
4,id_0000037,S,94,949,교회,기독교계통종교활동,종교서비스
...,...,...,...,...,...,...,...
100391,id_0999945,S,95,952,사업장에서,고객요청으로,자동차경정비
100392,id_0999958,S,94,941,지부에서,회원들의 친목도모,회원관리
100393,id_0999967,S,96,961,미용실,일반인대상,미용
100394,id_0999979,S,95,951,매장에서,고객의 의뢰에의해,핸드폰수리


In [6]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000001,S,95,952,카센터에서 자동차부분정비 타이어오일교환
1,id_0000017,S,95,952,사업장에서 고객요청으로 자동차세차업
2,id_0000029,S,96,961,미용실에서 일반인대상 컷.파마
3,id_0000034,S,96,969,세탁소에서 고객의 의뢰를 받아 가정용 세탁물 세탁
4,id_0000037,S,94,949,교회 기독교계통종교활동 종교서비스
...,...,...,...,...,...
100391,id_0999945,S,95,952,사업장에서 고객요청으로 자동차경정비
100392,id_0999958,S,94,941,지부에서 회원들의 친목도모 회원관리
100393,id_0999967,S,96,961,미용실 일반인대상 미용
100394,id_0999979,S,95,951,매장에서 고객의 의뢰에의해 핸드폰수리


In [7]:
df['digit_3'].value_counts()

961    43040
949    21580
969    13170
952    12142
953     6911
941     1612
951     1508
942      433
Name: digit_3, dtype: int64

## 맞춤법 교정

In [8]:
!pip install git+https://github.com/ssut/py-hanspell

  Cloning https://github.com/ssut/py-hanspell to /tmp/pip-req-build-sphuv0js
  Running command git clone -q https://github.com/ssut/py-hanspell /tmp/pip-req-build-sphuv0js
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=d5d64310b528cf525fa4f995afe981e22c6bc88590ab001abe431f1c966914d7
  Stored in directory: /tmp/pip-ephem-wheel-cache-wouj1iup/wheels/31/c2/c7/ddbbd9a84ddb0a67c2e52e3324d4b7b47dcf9581af9cd784de
Successfully built py-hanspell


In [9]:
from hanspell import spell_checker

temp_result=[]
for sentence in df['text']:
  try:
    result=spell_checker.check(sentence)
    temp_result.append(result.checked)
  except:
    temp_result.append(sentence)

df['text']=pd.DataFrame(temp_result)  

In [10]:
df.to_csv('df_S_spellcheck.csv', index=False, encoding='utf-8-sig')